In [47]:
import tensorflow as tf
import os, optuna
from tensorflow import keras
from dotenv import load_dotenv
load_dotenv(dotenv_path="local_paths.env")

os.chdir(os.getenv("LOCAL_REPO_DIR"))
K = keras.backend
from src.models.scheduler import OneCycleSchedulerNoMom

TypeError: chdir: path should be string, bytes, os.PathLike or integer, not NoneType

In [2]:
IMG_HEIGHT = 64 * 2
IMG_WIDTH = 48 * 2
BATCH_SIZE = 32
EPOCH = 50
DATA_DIR = os.getenv("LOCAL_DATA_DIR")

In [31]:
train_set, val_set = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.3,
    subset="both",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
class_names = train_set.class_names
num_classes = len(train_set.class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
test_set = val_set.shuffle(buffer_size=512).take(2000//BATCH_SIZE)
val_set = val_set.skip(2000//BATCH_SIZE)

Found 15349 files belonging to 3 classes.
Using 10745 files for training.
Using 4604 files for validation.


In [32]:
TRAIN_SIZE = 10745
TEST_SIZE = BATCH_SIZE * 100

In [33]:
loaded_study = optuna.load_study(
    study_name="resnet_onecycle_drop_sgd",
    storage="sqlite:///models/db_maize_models.sqlite3",
)

In [38]:
# Architecture Hyperparameters
size_dense = loaded_study.best_params["size_dense"]
activation_function = loaded_study.best_params["activation_function"]
dropout_rate = loaded_study.best_params["dropout_rate"]
# Input layers
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation=activation_function
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(
    conv_in
)
# R1
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
if activation_function == "relu":
    relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
else:
    relu_r1_1 = keras.layers.ELU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
if activation_function == "relu":
    relu_r1_2 = keras.layers.ReLU()(skip_r1)
else:
    relu_r1_2 = keras.layers.ELU()(skip_r1)
# R2
conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_2)
bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
if activation_function == "relu":
    relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
else:
    relu_r2_1 = keras.layers.ELU()(bn_r2_1)
conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r2_1)
bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
if activation_function == "relu":
    relu_r2_2 = keras.layers.ReLU()(skip_r2)
else:
    relu_r2_2 = keras.layers.ELU()(skip_r2)
# R3
conv_r3_skip = keras.layers.Conv2D(128, 1, 2, padding="same")(relu_r2_2)
conv_r3_1 = keras.layers.Conv2D(128, 3, 2, padding="same")(relu_r2_2)
bn_r3_1 = keras.layers.BatchNormalization()(conv_r3_1)
if activation_function == "relu":
    relu_r3_1 = keras.layers.ReLU()(bn_r3_1)
else:
    relu_r3_1 = keras.layers.ELU()(bn_r3_1)
conv_r3_2 = keras.layers.Conv2D(128, 3, 1, padding="same")(relu_r3_1)
bn_r3_2 = keras.layers.BatchNormalization()(conv_r3_2)
skip_r3 = keras.layers.Add()([bn_r3_2, conv_r3_skip])
if activation_function == "relu":
    relu_r3_2 = keras.layers.ReLU()(skip_r3)
else:
    relu_r3_2 = keras.layers.ELU()(skip_r3)
# Output layers
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r3_2)
dense_out = keras.layers.Dense(size_dense, activation=activation_function)(pool_out)
dropout = keras.layers.Dropout(dropout_rate)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
# Model
model = keras.Model(inputs=input, outputs=output)
# Fitting model

max_lr = loaded_study.best_params["max_lr"]
start_lr_prop = loaded_study.best_params["start_lr_prop"]
last_lr = loaded_study.best_params["last_lr"]
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=max_lr,
    start_lr=start_lr_prop * max_lr,
    last_lr=last_lr,
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True,
    start_from_epoch=20,
    monitor="val_accuracy",
    mode="max",
)

model.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

h = model.fit(
    train_set,
    validation_data=test_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
    verbose = 2
)

Epoch 1/50
336/336 - 53s - loss: 0.3442 - accuracy: 0.8632 - val_loss: 1.6470 - val_accuracy: 0.6369 - 53s/epoch - 159ms/step
Epoch 2/50
336/336 - 46s - loss: 0.2346 - accuracy: 0.9160 - val_loss: 0.4919 - val_accuracy: 0.7863 - 46s/epoch - 136ms/step
Epoch 3/50
336/336 - 45s - loss: 0.1951 - accuracy: 0.9321 - val_loss: 0.3798 - val_accuracy: 0.8478 - 45s/epoch - 134ms/step
Epoch 4/50
336/336 - 45s - loss: 0.1715 - accuracy: 0.9409 - val_loss: 1.1461 - val_accuracy: 0.7288 - 45s/epoch - 133ms/step
Epoch 5/50
336/336 - 45s - loss: 0.1491 - accuracy: 0.9480 - val_loss: 1.8696 - val_accuracy: 0.6621 - 45s/epoch - 134ms/step
Epoch 6/50
336/336 - 45s - loss: 0.1309 - accuracy: 0.9551 - val_loss: 1.9993 - val_accuracy: 0.6606 - 45s/epoch - 133ms/step
Epoch 7/50
336/336 - 45s - loss: 0.1163 - accuracy: 0.9603 - val_loss: 2.3317 - val_accuracy: 0.6470 - 45s/epoch - 134ms/step
Epoch 8/50
336/336 - 45s - loss: 0.1074 - accuracy: 0.9627 - val_loss: 2.0528 - val_accuracy: 0.6925 - 45s/epoch - 135

In [42]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

In [44]:
y_val = np.concatenate([y for x, y in val_set])
y_val_pred = np.argmax(model.predict(val_set), axis=1)
f1 = f1_score(y_val, y_val_pred, average="macro")
acc = accuracy_score(y_val, y_val_pred)
print(f"F1 score: {np.round(f1, 2)}, Accuracy: {np.round(acc, 2)}")

82/82 [==============================] - 6s 54ms/step
F1 score: 0.98, Accuracy: 0.98


In [46]:
model.save("models/maize_model.h5")